# Descriptive tables

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/Mastertabelle_filtered.xlsx'

In [ ]:
df = pd.read_excel(data_path)

In [ ]:
df.head()

In [ ]:
df = df[df.Comment != 'Reanimation']

# Table 1
age, sex, day-time, HHO, medical category, detailed medical category, mission time, on-scene time

In [ ]:
df['Sex'] = df['Sverweis_Geschlecht'].map({'männlich': 0, 'weiblich': 1})
df['28 d Survival [y/n]_encoded'] = df['28 d Survival [y/n]'].map({'n': 0, 'y': 1})
df['ITN ass. Pneumonia [y/n]_encoded'] = df['ITN ass. Pneumonia [y/n]'].map({'n': 0, 'y': 1})

In [ ]:
# encode nighttime as 1 (18-6)
df['daytime'] = ((df['Zeit (Erstbefund)'].dt.hour >= 18) | (df['Zeit (Erstbefund)'].dt.hour < 6)).astype(int)

In [ ]:
def get_population_stats(input_df):
    population_df = pd.DataFrame()
    population_df['n_patients'] = [input_df.shape[0]]
    
    population_df['age_median'] = input_df['Alter  (Patient)'].median()
    population_df['age_q1'] = input_df['Alter  (Patient)'].quantile(0.25)
    population_df['age_q3'] = input_df['Alter  (Patient)'].quantile(0.75)
    population_df['age_str'] = f'{population_df.age_median.iloc[0]:.0f} ({population_df.age_q1.iloc[0]:.0f}-{population_df.age_q3.iloc[0]:.0f})'
    
    population_df['n_female'] = input_df.Sex.sum()
    population_df['p_female'] = input_df.Sex.sum() / input_df.shape[0]
    population_df['female_str'] = f'{int(population_df.n_female.iloc[0])} ({population_df.p_female.iloc[0]*100:.1f}%)'
    
    population_df['n_nighttime'] = input_df.daytime.sum()
    population_df['p_nighttime'] = input_df.daytime.sum() / input_df.shape[0]
    population_df['nighttime_str'] = f'{population_df.n_nighttime.iloc[0]} ({population_df.p_nighttime.iloc[0]*100:.1f}%)'
    
    return population_df

In [ ]:
medical_pop_df = get_population_stats(df[df['Einteilung (Hauptdiagnose)'] == 'Krankheit'])
trauma_pop_df = get_population_stats(df[df['Einteilung (Hauptdiagnose)'] == 'Unfall'])
overall_pop_df = get_population_stats(df)

In [ ]:
full_pop_df = pd.concat([overall_pop_df, medical_pop_df, trauma_pop_df], keys=['overall', 'medical', 'trauma'])

In [ ]:
full_pop_df = full_pop_df.droplevel(1).T

In [ ]:
str_pop_df = full_pop_df.loc[['n_patients', 'age_str', 'female_str', 'nighttime_str']]

In [ ]:
str_pop_df.index = ['Number of patients', 'Age', 'Sex (Female)', 'Nighttime Operation']
str_pop_df.rename(columns={'medical': 'Medical', 'trauma': 'Trauma', 'overall': 'Overall population'}, inplace=True)

In [ ]:
str_pop_df

In [ ]:
# str_pop_df.to_csv('/Users/jk1/Downloads/table_1.csv')

## Outcome table

In [ ]:
outcomes_of_interest = [
    "ICU [d]",
    "Ventilator [d]",
    "Hospital stay [d]",
    "28 d Survival [y/n]",
    "ITN ass. Pneumonia [y/n]"
    ]

In [ ]:
time_columns =["ICU [d]",
    "Ventilator [d]",
    "Hospital stay [d]"]
# in every column, if value ends with +, remove it
for col in time_columns:
    df[col] = pd.to_numeric(df[col].apply(lambda x: str(x).replace("+", '')), errors='coerce')

In [ ]:
def get_population_outcomes(input_df):
    outcomes_df = pd.DataFrame()
    
    outcomes_df['icu_los_median'] = [input_df["ICU [d]"].median()]
    outcomes_df['icu_los_q1'] = input_df["ICU [d]"].quantile(0.25)
    outcomes_df['icu_los_q3'] = input_df["ICU [d]"].quantile(0.75)
    outcomes_df['icu_los_str'] = f'{outcomes_df.icu_los_median.iloc[0]:.0f} ({outcomes_df.icu_los_q1.iloc[0]:.0f}-{outcomes_df.icu_los_q3.iloc[0]:.0f})'
    
       # Ventilator Days
    outcomes_df['vent_days_median'] = input_df["Ventilator [d]"].median()
    outcomes_df['vent_days_q1'] = input_df["Ventilator [d]"].quantile(0.25)
    outcomes_df['vent_days_q3'] = input_df["Ventilator [d]"].quantile(0.75)
    outcomes_df['vent_days_str'] = f'{outcomes_df.vent_days_median.iloc[0]:.0f} ({outcomes_df.vent_days_q1.iloc[0]:.0f}-{outcomes_df.vent_days_q3.iloc[0]:.0f})'

    # Hospital Stay
    outcomes_df['hospital_stay_median'] = input_df["Hospital stay [d]"].median()
    outcomes_df['hospital_stay_q1'] = input_df["Hospital stay [d]"].quantile(0.25)
    outcomes_df['hospital_stay_q3'] = input_df["Hospital stay [d]"].quantile(0.75)
    outcomes_df['hospital_stay_str'] = f'{outcomes_df.hospital_stay_median.iloc[0]:.0f} ({outcomes_df.hospital_stay_q1.iloc[0]:.0f}-{outcomes_df.hospital_stay_q3.iloc[0]:.0f})'
    
    # 28 d Survival
    outcomes_df['n_survivors'] = input_df['28 d Survival [y/n]'].map({'n': 0, 'y': 1}).sum()
    outcomes_df['p_survivors'] = input_df['28 d Survival [y/n]'].map({'n': 0, 'y': 1}).sum() / input_df.shape[0]
    outcomes_df['survivors_str'] = f'{outcomes_df.n_survivors.iloc[0]} ({outcomes_df.p_survivors.iloc[0]*100:.1f}%)'
    
    # pneumonia
    outcomes_df['n_pneumonia_yes'] = input_df["ITN ass. Pneumonia [y/n]"].map({'n': 0, 'y': 1}).sum()
    outcomes_df['p_pneumonia_yes'] = outcomes_df['n_pneumonia_yes'] / input_df.shape[0]
    outcomes_df['pneumonia_str'] = f'{outcomes_df.n_pneumonia_yes.iloc[0]} ({outcomes_df.p_pneumonia_yes.iloc[0]*100:.1f}%)'
    
    return outcomes_df



In [ ]:
medical_pop_outcomes_df = get_population_outcomes(df[df['Einteilung (Hauptdiagnose)'] == 'Krankheit'])
trauma_pop_outcomes_df = get_population_outcomes(df[df['Einteilung (Hauptdiagnose)'] == 'Unfall'])
overall_outcomes_df = get_population_outcomes(df)

In [ ]:
full_outcomes_df = pd.concat([overall_outcomes_df, medical_pop_outcomes_df, trauma_pop_outcomes_df], keys=['overall', 'medical', 'trauma'])

In [ ]:
full_outcomes_df

In [ ]:
full_outcomes_df = full_outcomes_df.droplevel(1).T

In [ ]:
# str_pop_df = full_pop_df.loc[['n_patients', 'age_str', 'female_str', 'nighttime_str']]
# str_pop_df.index = ['Number of patients', 'Age', 'Sex (Female)', 'Nighttime Operation']
# str_pop_df.rename(columns={'medical': 'Medical', 'trauma': 'Trauma', 'overall': 'Overall population'}, inplace=True)

str_outcomes_df = full_outcomes_df.loc[['icu_los_str', 'vent_days_str', 'hospital_stay_str', 'survivors_str', 'pneumonia_str']]
str_outcomes_df.index = ['ICU stay (d)', 'Ventilator Days (d)', 'Hospital stay (d)', 'Survival (28d)', 'Ventilator-associated pneumonia']
str_outcomes_df.rename(columns={'medical': 'Medical', 'trauma': 'Trauma', 'overall': 'Overall population'}, inplace=True)

In [ ]:
str_outcomes_df

In [ ]:
# str_outcomes_df.to_csv('/Users/jk1/Downloads/table_2.csv')

## Airway management table

In [ ]:
rows_to_be_swapped = [16, 35, 44, 130, 226]
airway_management_df = df['Detailliert (Atemweg-Management)']
airway_management_df = airway_management_df.str.split(';', expand=True)
airway_management_df.columns = ['main_method', 'auxillary']

# print warning that rows are being swapped
print('Rows being swapped:', rows_to_be_swapped)
print('Before swapping:', airway_management_df.loc[rows_to_be_swapped])
print('Verify that the swapping is correct')

# for these rows 1st attempt and 2nd attempt should be swapped
for row in rows_to_be_swapped:
    temp = airway_management_df.loc[row, 'main_method']
    airway_management_df.loc[row, 'main_method'] = airway_management_df.loc[row, 'auxillary']
    airway_management_df.loc[row, 'auxillary'] = temp
    
print('After swapping:', airway_management_df.loc[rows_to_be_swapped])
        

In [ ]:
# join with main df 
df = pd.concat([df, airway_management_df], axis=1)

In [ ]:
def get_intubation_df(input_df):       
    airway_management_df = input_df['main_method']
    # parse csv from every row with following columns:
    # Intubationsart, Tubusgrösse, Zahnreihe, Versuche, Erfolg, CL-Score, Stimmbänder, vorbestehend, Intubierende Person, Uhrzeit
    airway_management_df = airway_management_df.str.split(',', expand=True)
    airway_management_df.columns = ['Intubation type', 'Tube size', 'Dental row', 'Attempts', 'Success', 'CL-Score',
                                    'Vocal cords', 'Pre-existing', 'Intubator', 'Time']
        
    intubation_df = pd.DataFrame()
    intubation_df['n_patients'] = [input_df.shape[0]]
    
    # number of attempts
    # set '-' to nan
    airway_management_df['Attempts'] = airway_management_df['Attempts'].replace(' -', np.nan)
    intubation_df['attempts_median'] = [airway_management_df['Attempts'].astype(float).median()]
    intubation_df['attempts_q1'] = airway_management_df['Attempts'].astype(float).quantile(0.25)
    intubation_df['attempts_q3'] = airway_management_df['Attempts'].astype(float).quantile(0.75)
    intubation_df['attempts_n_missing'] = airway_management_df['Attempts'].isna().sum()
    intubation_df['attempts_str'] = f'{intubation_df.attempts_median.iloc[0]:.0f} ({intubation_df.attempts_q1.iloc[0]:.0f}-{intubation_df.attempts_q3.iloc[0]:.0f}) [missing: {intubation_df.attempts_n_missing.iloc[0]}]'
    
    # CL-Score
    # current values ' -', ' CL I', ' CL II', ' CL III', ' CL IV'
    # convert to nan, 1, 2, 3, 4
    airway_management_df['CL-Score'] = airway_management_df['CL-Score'].replace(' -', np.nan)
    airway_management_df['CL-Score'] = airway_management_df['CL-Score'].replace(' CL I', 1)
    airway_management_df['CL-Score'] = airway_management_df['CL-Score'].replace(' CL II', 2)
    airway_management_df['CL-Score'] = airway_management_df['CL-Score'].replace(' CL III', 3)
    airway_management_df['CL-Score'] = airway_management_df['CL-Score'].replace(' CL IV', 4)
    intubation_df['cl_score_median'] = airway_management_df['CL-Score'].median()
    intubation_df['cl_score_q1'] = airway_management_df['CL-Score'].quantile(0.25)
    intubation_df['cl_score_q3'] = airway_management_df['CL-Score'].quantile(0.75)
    intubation_df['cl_score_n_missing'] = airway_management_df['CL-Score'].isna().sum()
    intubation_df['cl_score_str'] = f'{intubation_df.cl_score_median.iloc[0]:.0f} ({intubation_df.cl_score_q1.iloc[0]:.0f}-{intubation_df.cl_score_q3.iloc[0]:.0f}) [missing: {intubation_df.cl_score_n_missing.iloc[0]}]'
    
    return intubation_df

In [ ]:
airway_management_df['CL-Score'].unique()

In [ ]:
medical_intubation_df = get_intubation_df(df[df['Einteilung (Hauptdiagnose)'] == 'Krankheit'])
trauma_intubation_df = get_intubation_df(df[df['Einteilung (Hauptdiagnose)'] == 'Unfall'])
overall_intubation_df = get_intubation_df(df)

In [ ]:
full_airway_df = pd.concat([overall_intubation_df, medical_intubation_df, trauma_intubation_df], keys=['overall', 'medical', 'trauma'])
full_airway_df

In [ ]:
full_airway_df = full_airway_df.droplevel(1).T

In [ ]:
str_airway_df = full_airway_df.loc[['n_patients', 'attempts_str', 'cl_score_str']]
str_airway_df.index = ['Number of patients', 'Number of attempts', 'Cormack-Lehane Score']
str_airway_df.rename(columns={'medical': 'Medical', 'trauma': 'Trauma', 'overall': 'Overall population'}, inplace=True)

In [ ]:
str_airway_df

In [ ]:
# str_airway_df.to_csv('/Users/jk1/Downloads/table_3.csv')